# 分析テーマ
1. RFM分析
2. メールの開封率予測
3. キャンペーンの効果検証

> 詳細は後ほど追記

## データの読み込み

In [20]:
# 必要なライブラリのインポート
import pandas as pd

# データのPATHを格納
user_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/user_master_2022.csv'
purchase_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/purchase_master_2022.csv'
mail_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/mail_master_202211.csv'
coupon_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/coupon_master_202301.csv'

# 各データをデータフレームに格納
df_user = pd.read_csv(user_path)
df_purchase = pd.read_csv(purchase_path)
df_mail = pd.read_csv(mail_path)
df_coupon = pd.read_csv(coupon_path)

# ユーザーデータの中身を確認
print('--------------------')
print('ユーザーデータ')
print('--------------------')
display(df_user.head())

# 購買データの中身を確認
print('--------------------')
print('購買データ')
print('--------------------')
display(df_purchase.head())

# メールデータの中身を確認
print('--------------------')
print('メールデータ')
print('--------------------')
display(df_mail.head())

# クーポンデータの中身を確認
print('--------------------')
print('クーポンデータ')
print('--------------------')
display(df_coupon.head())

--------------------
ユーザーデータ
--------------------


,id,gender,age,first_purchase_date,last_purchase_date
0,1,女性,37,2022-02-27,2022-12-17
1,2,男性,61,2022-07-04,2022-12-10
2,3,女性,41,2022-03-10,2022-10-03
3,4,女性,55,2021-09-11,2022-10-08
4,5,女性,52,2022-03-15,2022-11-06


--------------------
購買データ
--------------------


,id,purchase_price,purchase_date
0,525,10000,2021-04-03
1,507,10000,2021-04-06
2,525,3000,2021-04-16
3,550,3000,2021-04-18
4,437,5000,2021-04-19


--------------------
メールデータ
--------------------


,id,mail_opened
0,1,1
1,2,0
2,3,1
3,4,1
4,5,1


--------------------
クーポンデータ
--------------------


,id,coupon_dummy,purchase_dummy,purchase_price
0,1,0,0,0
1,2,0,1,3000
2,3,0,0,0
3,4,0,1,3000
4,5,0,0,0


# RFM分析

In [29]:
# 必要なライブラリをインポート
import datetime
import pandas as pd

# 必要なデータを再度データフレームに格納（一度やっていれば不要）
df_purchase = pd.read_csv(purchase_path)

# 日付カラムが文字列データとなっているので，日付データに変換（一度やっていれば不要）
df_purchase['purchase_date'] = pd.to_datetime(df_purchase['purchase_date'])

# IDベースでデータを集計
df = df_purchase.groupby('id', as_index=False).agg(rc=('purchase_date', 'max'), 
                                                   fq=('purchase_date', 'count'), 
                                                   mo=('purchase_price', 'sum'))

# recencyを評価しやすくするために，rcカラムを2022年12月31日との差分に変更
base_date = datetime.datetime(2022, 12, 31)
df['rc'] = df['rc'].apply(lambda x: base_date-x).dt.days

# 各セグメントの基準を設定
rc_median = df['rc'].median()
fq_median = df['fq'].median()
mo_median = df['mo'].median()

# 優良顧客（R小, F・M大）
df1 = df[(df['rc']<rc_median)&(df['fq']>fq_median)&(df['mo']>mo_median)]
# 新規顧客・特定利用顧客（R小. F・M小）
df2 = df[(df['rc']<rc_median)&(df['fq']<fq_median)&(df['mo']<mo_median)]
# 過去の優良顧客（R大, F・M大）
df3 = df[(df['rc']>rc_median)&(df['fq']>fq_median)&(df['mo']>mo_median)]
# 優先度の低い顧客（R大, F・M小）
df4 = df[(df['rc']>rc_median)&(df['fq']<fq_median)&(df['mo']<mo_median)]

# それぞれのIDをリストに格納
id1_lst = df1['id'].to_list()
id2_lst = df2['id'].to_list()
id3_lst = df3['id'].to_list()
id4_lst = df4['id'].to_list()

# IDリストを出力
print(f"ID1リスト: {id1_lst}")
print(f"ID2リスト: {id2_lst}")
print(f"ID3リスト: {id3_lst}")
print(f"ID4リスト: {id4_lst}")

ID1リスト: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 242, 245, 293, 333, 365]
ID2リスト: [39, 201, 202, 203, 204, 205, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

# メールの開封率予測

### 1. データの準備

In [16]:
# 必要なライブラリをインポート
import datetime
import pandas as pd

# 必要なデータを再度データフレームに格納（一度やっていれば不要）
df_user = pd.read_csv(user_path)
df_purchase = pd.read_csv(purchase_path)
df_mail = pd.read_csv(mail_path)

# 日付カラムが文字列データとなっているので，日付データに変換（一度やっていれば不要）
df_user['first_purchase_date'] = pd.to_datetime(df_user['first_purchase_date'])
df_user['last_purchase_date'] = pd.to_datetime(df_user['last_purchase_date'])
df_purchase['purchase_date'] = pd.to_datetime(df_purchase['purchase_date'])

# メール送信日（2022年11月1日）を格納
send_date = datetime.datetime(2022, 11, 1)

# ユーザーデータをメール送信日以前のものに絞り込む
df_user_202211 = df_user[df_user['first_purchase_date']<send_date]
df_user_202211 = df_user_202211[['id', 'gender', 'age']]

# 購買データをメール送信日以前のものに絞り込んでIDベースで集計する
df_purchase_202211 = df_purchase[df_purchase['purchase_date']<send_date]
df_purchase_202211 = df_purchase_202211.groupby('id', as_index=False).agg(rc=('purchase_date', 'max'), 
                                                                          fq=('purchase_date', 'count'), 
                                                                          mo=('purchase_price', 'sum'))

# データを結合
## ユーザーデータと購買データを結合
df = df_user_202211.merge(df_purchase_202211, how='left', on='id')
## 上記のデータとメールデータを結合
df = df.merge(df_mail, how='left', on='id')

# モデル構築のためのデータの前処理
## genderカラムを数値データ（男性1, 女性0）に変換
df['gender'] = df['gender'].apply(lambda x: 1 if x=='男性' else 0)
## frequencyカラムを生成
df['rc'] = df['rc'].apply(lambda x: send_date-x).dt.days
## 各特徴量（age, recency, frequency, monetary）を正規化
df['age'] = (df['age'].max()-df['age']) / df['age'].max()
df['rc'] = (df['rc'].max()-df['rc']) / df['rc'].max()
df['fq'] = (df['fq'].max()-df['fq']) / df['fq'].max()
df['mo'] = (df['mo'].max()-df['mo']) / df['mo'].max()

# 目的変数と特徴量の設定
X = df[['gender', 'age', 'rc', 'fq', 'mo']]
y = df['mail_opened']

# 特徴量の確認
display(X.head())
print('---------------------')
print('メール開封ダミー')
print('---------------------')
display(y.head())

,gender,age,rc,fq,mo
0,0,0.447761,0.973244,0.391304,0.638889
1,1,0.089552,0.969900,0.739130,0.680556
2,0,0.388060,0.903010,0.478261,0.479167
3,0,0.179104,0.919732,0.130435,0.173611
4,0,0.223881,0.943144,0.478261,0.513889


---------------------
メール開封ダミー
---------------------


0    1
1    0
2    1
3    1
4    1
Name: mail_opened, dtype: int64

### 2. テストデータと訓練データに分割する

In [8]:
# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### 3・4. モデルの選定および訓練データの学習

In [9]:
# 必要なライブラリをインポート
from sklearn.ensemble import RandomForestClassifier

# 訓練データの学習
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

### 5. モデルの評価

In [10]:
# 必要なライブラリをインポート
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# テストデータを用いて予測値を算出
y_pred = clf.predict(X_test)

# モデルの評価
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"正解率: {accuracy*100:.1f}%")
print(f"適合率: {precision*100:.1f}%")
print(f"再現率: {recall*100:.1f}%")
print(f"F値: {f1*100:.1f}%")

正解率: 62.0%
適合率: 56.5%
再現率: 55.9%
F値: 56.2%


### モデルを利用してメール送信ユーザーリストを出力

In [17]:
# 実運用
send_date = datetime.datetime(2022, 12, 31)
df_user_202212 = df_user[['id', 'gender', 'age']]
df_purchase_202212 = df_purchase.groupby('id', as_index=False).agg(rc=('purchase_date', 'max'), 
                                                                   fq=('purchase_date', 'count'), 
                                                                   mo=('purchase_price', 'sum'))
# ユーザーデータと購買データを結合
df = df_user_202212.merge(df_purchase_202212, how='left', on='id')

# モデル構築のためのデータの前処理
## genderカラムを数値データ（男性1, 女性0）に変換
df['gender'] = df['gender'].apply(lambda x: 1 if x=='男性' else 0)
## frequencyカラムを生成
df['rc'] = df['rc'].apply(lambda x: send_date-x).dt.days

## 各特徴量の正規化
df['age'] = (df['age'].max()-df['age']) / df['age'].max()
df['rc'] = (df['rc'].max()-df['rc']) / df['rc'].max()
df['fq'] = (df['fq'].max()-df['fq']) / df['fq'].max()
df['mo'] = (df['mo'].max()-df['mo']) / df['mo'].max()

## 特徴量の設定
X = df[['gender', 'age', 'rc', 'fq', 'mo']]

# 予測
y_pred = clf.predict(X)
# 予測結果の格納
df['y_pred'] = y_pred

# メールを開封すると予測されたユーザーIDを出力
send_lst = df[df['y_pred']==1]['id'].tolist()
print(send_lst)

[1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 40, 43, 44, 45, 46, 47, 49, 51, 53, 55, 56, 58, 59, 60, 61, 65, 66, 67, 68, 69, 70, 73, 76, 77, 78, 79, 82, 83, 84, 85, 86, 87, 88, 90, 92, 93, 97, 98, 99, 101, 102, 104, 105, 106, 107, 109, 110, 111, 112, 114, 119, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 139, 140, 141, 142, 143, 144, 147, 148, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 174, 175, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 205, 208, 209, 211, 214, 216, 218, 223, 226, 227, 239, 241, 245, 251, 257, 258, 260, 264, 273, 275, 276, 278, 284, 285, 288, 290, 295, 303, 311, 314, 315, 316, 320, 322, 328, 334, 337, 338, 341, 342, 343, 346, 352, 354, 355, 356, 359, 360, 363, 372, 373, 374, 382, 383, 392, 397, 398, 403, 405, 409, 413, 416, 418, 419, 421, 427, 428, 429, 438, 44

## 効果検証

In [19]:
# 必要なライブラリをインポート（一度インポートしていれば不要）
import pandas as pd

# 必要なデータを再度データフレームに格納（一度やっていれば不要）
df_user = pd.read_csv(user_path)
df_coupon = pd.read_csv(coupon_path)

# 日付カラムが文字列データとなっているので，日付データに変換（一度やっていれば不要）
df_user['first_purchase_date'] = pd.to_datetime(df_user['first_purchase_date'])
df_user['last_purchase_date'] = pd.to_datetime(df_user['last_purchase_date'])

# ユーザーデータとクーポンデータを結合
df = df_user.merge(df_coupon, how='left', on='id')

# 最終購買日時が90日±10日間のデータを抽出
cutoff_date = datetime.datetime(2022, 10, 1)
td = datetime.timedelta(days=10)
lower_date = cutoff_date - td
upper_date = cutoff_date + td
df= df[(df['last_purchase_date']>lower_date)&(df['last_purchase_date']<upper_date)]

# 最終購買日時が90日±10日間のデータの購入率を算出
df[['coupon_dummy', 'purchase_dummy']].groupby('coupon_dummy').mean()

,purchase_dummy
coupon_dummy,
0,0.277778
1,0.400000
